<a href="https://colab.research.google.com/github/euna-jeong20/Intro_Recommender_System/blob/main/Word2vec%EC%9D%84_%EC%9D%B4%EC%9A%A9%ED%95%9C_%EC%B6%94%EC%B2%9C%EC%8B%9C%EC%8A%A4%ED%85%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* 컨텐츠 기반 모델

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim   #WORD2VEC의 알고리즘을 호출해주는 패키지

In [ ]:
path = '/content/drive/MyDrive/recommender_system/movies/'

In [ ]:
movie = pd.read_csv(path + 'ratings.csv', low_memory=False)
movie.head(2)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179


In [ ]:
#시간순으로 정렬
movie = movie.sort_values(by='timestamp', ascending=True).reset_index(drop=True)
movie.head()

,userId,movieId,rating,timestamp
0,383,21,3.0,789652009
1,383,47,5.0,789652009
2,383,1079,3.0,789652009
3,409,85,5.0,828212412
4,409,35,4.0,828212412


In [ ]:
#영화의 metadata를 불러와서 movieID에 맞는 TITLE을 구한다.
meta = pd.read_csv(path + 'movies_metadata.csv', low_memory=False)
meta.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [ ]:
meta.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [ ]:
meta = meta.rename(columns={'id' : 'movieId'})      # 둘의 col명을 맞춰주기위해
movie['movieId'] = movie['movieId'].astype(str)     # str형태로 바꿔준다
meta['movieId'] = meta['movieId'].astype(str)       # str형태로 바꿔준다

movie = pd.merge(movie, meta[['movieId', 'original_title']], how='left', on='movieId')

In [ ]:
movie.head()

,userId,movieId,rating,timestamp,original_title
0,383,21,3.0,789652009,The Endless Summer
1,383,47,5.0,789652009,NaN
2,383,1079,3.0,789652009,NaN
3,409,85,5.0,828212412,Raiders of the Lost Ark
4,409,35,4.0,828212412,The Simpsons Movie


In [ ]:
movie = movie[movie['original_title'].notnull()].reset_index(drop=True)     #'original_title'의 결측치 제거

In [ ]:
agg = movie.groupby(['userId'])['original_title'].agg({'unique'})
agg.head()

,unique
userId,
1,"[Jay and Silent Bob Strike Back, Vivement dima..."
2,"[48 Hrs., The Hours, Terminator 3: Rise of the..."
3,"[300, The Killing, Shortbus, Finding Neverland..."
4,"[David, The Wedding Planner, Casablanca, Sleep..."
5,"[Gleaming the Cube, Cool Hand Luke, Hidalgo, U..."


* 한 사람마다 영화를 뭘 봤고 무슨 순서로 봤는지 알 수 있다.

In [ ]:
movie['original_title'].unique()

array(['The Endless Summer', 'Raiders of the Lost Ark',
       'The Simpsons Movie', ..., 'The Lonedale Operator',
       'Violeta se fue a los cielos', 'To Kill a Priest'], dtype=object)

In [ ]:
# str로 변경
sentence = []

for user_sentence in agg['unique'].values:
    sentence.append(list(map(str, user_sentence)))

In [ ]:
sentence[0]

['Jay and Silent Bob Strike Back',
 'Vivement dimanche!',
 'Rocky III',
 'American Pie',
 'My Tutor',
 'Greed']

In [ ]:
from gensim.models import Word2Vec

embedding_model = Word2Vec(sentence, size=20, window=5,
                           min_count=1, workers=4, iter=200, sg=1)

* size = 워드 벡터의 특징 값. 즉, 임베딩 된 벡터의 차원.
* window = 양 옆 주변단어들을 몇개 볼지
* min_count = 단어 최소 빈도 수 제한 (빈도가 적은 단어들은 학습하지 않는다.)
* workers = 학습을 위한 프로세스 수
* sg = 0은 CBOW, 1은 Skip-gram.

In [ ]:
embedding_model.wv.most_similar(positive=['Spider-Man 2'], topn=10)     #wv는 가중치 벡터

[('Snow Cake', 0.8830479383468628),
 ('Rumor Has It...', 0.8176966905593872),
 ('Conquest of the Planet of the Apes', 0.7685685157775879),
 ('Licence to Kill', 0.7657096982002258),
 ('To Die For', 0.7570158839225769),
 ('National Treasure', 0.7389312982559204),
 ('Dark Star', 0.7324922680854797),
 ('The Godfather', 0.7314088344573975),
 ('Heavenly Creatures', 0.7295575141906738),
 ('Harry Potter and the Prisoner of Azkaban', 0.7289674878120422)]